In [1]:
# Step 1: Upload the dataset file
from google.colab import files
uploaded = files.upload()

# Step 2: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Step 3: Load and clean the dataset
file_path = 'Dataset.csv'  # Use the file name you uploaded
data = pd.read_csv(file_path)

# Drop rows with missing values
data_cleaned = data.dropna()

# Encode the labels
label_encoder = LabelEncoder()
data_cleaned['Encoded_Label'] = label_encoder.fit_transform(data_cleaned['Label'])

# Step 4: Split the data
X_train, X_test, y_train, y_test = train_test_split(
    data_cleaned['Sentence'], data_cleaned['Encoded_Label'], test_size=0.2, random_state=42)

# Step 5: Create a pipeline with TF-IDF vectorizer and SVM model
pipeline = make_pipeline(TfidfVectorizer(), LinearSVC())

# Train the model
pipeline.fit(X_train, y_train)

# Step 6: Predict on the test set and calculate metrics
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

# Display accuracy and classification report
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)

# Step 7: Save the trained model and label encoder
pipeline_path = 'sentence_classifier_pipeline.pkl'
label_encoder_path = 'label_encoder.pkl'
joblib.dump(pipeline, pipeline_path)
joblib.dump(label_encoder, label_encoder_path)

print("Model and label encoder saved as:", pipeline_path, "and", label_encoder_path)


Saving Dataset.csv to Dataset.csv


<ipython-input-1-b38ab03187eb>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Encoded_Label'] = label_encoder.fit_transform(data_cleaned['Label'])
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Model Accuracy: 78.95%
Classification Report:
                                        precision    recall  f1-score   support

                         Availability       1.00      0.50      0.67         2
Compatibility with Microsoft Products       1.00      1.00      1.00         2
                    Disaster Recovery       1.00      0.50      0.67         2
                        Establishment       0.75      1.00      0.86         6
                         Market Share       0.00      0.00      0.00         1
                Offline Data Transfer       1.00      1.00      1.00         2
                       Pricing Models       1.00      0.67      0.80         3
              Storage Service Options       0.33      1.00      0.50         1

                             accuracy                           0.79        19
                            macro avg       0.76      0.71      0.69        19
                         weighted avg       0.83      0.79      0.77        19

Mo

In [5]:
# Step 1: Upload the PDF, model, and label encoder files
from google.colab import files

# Upload the PDF file and previously saved model files
uploaded = files.upload()

# Step 2: Install PyMuPDF (fitz) to handle PDF extraction
!pip install pymupdf

# Step 3: Import necessary libraries
import fitz  # PyMuPDF
import joblib
import re

# Step 4: Load the trained model and label encoder
pipeline = joblib.load('sentence_classifier_pipeline.pkl')  # Model file
label_encoder = joblib.load('label_encoder.pkl')  # Label encoder file

# Function to preprocess text and split it into sentences
def preprocess_text(text):
    # Use regular expressions to split text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Step 5: Extract text from the uploaded PDF
pdf_path = 'SRS.pdf'  # Replace with the actual filename of your PDF
text = ""
with fitz.open(pdf_path) as pdf:
    for page in pdf:
        text += page.get_text()

# Step 6: Preprocess the extracted text into individual sentences
sentences = preprocess_text(text)

# Step 7: Classify each sentence and print the predicted labels
print("Classification Results:\n")
for sentence in sentences:
    # Predict the label for each sentence
    label_code = pipeline.predict([sentence])[0]
    label = label_encoder.inverse_transform([label_code])[0]
    print(f"Sentence: {sentence}")
    print(f"Predicted Label: {label}\n")


Saving SRS.pdf to SRS.pdf
Classification Results:

Sentence: Software Requirements Specification (SRS) Document 
 
1.
Predicted Label: Pricing Models

Sentence: Introduction 
This Software Requirements Specification (SRS) document provides detailed information about 
the cloud provider’s services, focusing on core aspects such as its establishment, service 
availability, pricing models, offline data transfer capabilities, compatibility with Microsoft 
products, disaster recovery protocols, market share, and storage service options.
Predicted Label: Disaster Recovery

Sentence: This document 
is intended for stakeholders involved in evaluating or implementing cloud services.
Predicted Label: Establishment

Sentence: 2.
Predicted Label: Establishment

Sentence: System Overview 
The cloud platform is designed to provide reliable, high-availability services to businesses and 
enterprises.
Predicted Label: Availability

Sentence: It offers flexible pricing models, seamless integration with 